In [1]:
import sys, os
from Deconvolution.BLADE import Framework
import numpy as np
from numpy import transpose as t
import itertools
import pickle
from scipy.optimize import nnls
from sklearn.svm import SVR
from sklearn.svm import NuSVR

from sklearn.metrics import mean_squared_error as mse
import pandas as pd

# modules for visualization
import qgrid
from matplotlib import pyplot as plt
import seaborn as sns

# pseudobulk analysis


In [4]:
scRNAref =  pd.read_csv("/home/cke/Puram/Puram_pseudobulk_scRNAref_signature.tsv",index_col=0)

In [5]:
scRNAref

,HN28_P15_D06_S330_comb,HN28_P6_G05_S173_comb,HN26_P14_D11_S239_comb,HN26_P14_H05_S281_comb,HN26_P25_H09_S189_comb,HN26_P14_H06_S282_comb,HN25_P25_C04_S316_comb,HN26_P25_A11_S107_comb,HN26_P25_C09_S129_comb,HNSCC26_P24_H05_S377_comb,...,HNSCC17_P4_B04_S112_comb,HNSCC17_P4_C09_S129_comb,HNSCC17_P4_F06_S162_comb,HNSCC17_P4_H03_S183_comb,HNSCC20_P3_F09_S69_comb,HNSCC17_P4_G12_S180_comb,HNSCC20_P13_C05_S221_comb,HNSCC17_P4_C12_S132_comb,HNSCC20_P3_H08_S92_comb,HNSCC20_P3_G06_S78_comb
AREG,0.00000,0.00000,0.00000,0.00000,0.0000,0.44042,5.6999,0.00000,2.47270,2.78490,...,0.00000,0.30918,0.00000,0.000000,0.00000,4.96160,0.00000,3.926100,0.00000,0.00000
QPCTL,0.24853,0.72247,0.22280,0.15056,0.2302,0.22404,1.9358,0.35276,0.12168,1.27740,...,0.45838,0.75360,0.73898,0.000000,0.35840,0.86790,1.61960,0.280960,0.43082,0.66266
CNN3,7.04550,3.70750,5.28750,7.04200,4.1371,2.69890,0.0000,6.99060,3.86620,4.09200,...,4.88150,5.65310,1.87110,2.019300,0.42008,6.03100,0.00000,3.119200,0.00000,0.84157
MTRNR2L6,6.06600,6.41100,6.43460,5.23330,6.7382,4.90330,5.2151,6.41070,6.96490,6.95410,...,4.25600,3.71810,3.73100,2.707300,3.06470,5.49770,5.08650,3.745200,2.69670,2.61820
PTPN1,0.00000,0.00000,1.47510,0.00000,0.0000,2.43030,1.4163,0.29160,0.00000,0.00000,...,0.00000,1.25280,0.00000,0.000000,0.00000,4.95640,0.00000,0.875780,1.45000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOXP1,0.16993,3.49540,0.22651,4.82770,2.6567,2.64820,2.8466,2.92850,1.76980,0.00000,...,0.00000,2.49670,2.22530,0.000000,0.31034,0.00000,0.00000,0.453120,0.00000,0.00000
RABAC1,6.13160,5.22700,3.82970,0.00000,2.8266,2.66790,6.3580,5.63080,0.00000,4.64730,...,4.13770,4.50430,4.22170,0.000000,0.00000,4.62330,4.83470,3.048600,0.00000,5.19280
PTGS1,0.00000,0.00000,0.18777,0.00000,0.0000,0.02148,0.0000,0.00000,0.00000,0.00000,...,3.80180,2.66990,2.27410,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000
ZNF793,0.42438,0.99928,0.24853,0.41143,0.3115,0.64985,1.9054,0.53804,0.43936,0.57725,...,0.36065,0.31615,0.27501,0.063503,0.25822,0.59359,0.67265,0.044044,0.59264,0.33113


In [10]:
labels = pd.read_csv("/home/cke/Puram/PuramHNSCC_cell_category_mappedCelltypes_simple_symbol.csv",index_col=0)

In [13]:
dict_label = labels.to_dict()['cell_category']
dict_label

{'HN28_P15_D06_S330_comb': 'Fibroblast',
 'HN28_P6_G05_S173_comb': 'Fibroblast',
 'HN26_P14_D11_S239_comb': 'tumor',
 'HN26_P14_H05_S281_comb': 'Fibroblast',
 'HN26_P25_H09_S189_comb': 'tumor',
 'HN26_P14_H06_S282_comb': 'tumor',
 'HN25_P25_C04_S316_comb': 'tumor',
 'HN26_P25_A11_S107_comb': 'Fibroblast',
 'HN26_P25_C09_S129_comb': 'tumor',
 'HNSCC26_P24_H05_S377_comb': 'tumor',
 'HN25_P5_E10_S58_comb': 'Fibroblast',
 'HN25_P24_B02_S110_comb': 'tumor',
 'HN25_P25_C01_S313_comb': 'B cell',
 'HN28_P15_G06_S366_comb': 'Fibroblast',
 'HN26_P6_B06_S18_comb': 'Fibroblast',
 'HN25_P5_C05_S29_comb': 'Fibroblast',
 'HN28_P6_D09_S141_comb': 'Fibroblast',
 'HN26_P5_H01_S85_comb': 'tumor',
 'HN26_P13_D07_S139_comb': 'Fibroblast',
 'HN26_P5_E06_S54_comb': 'tumor',
 'HN26_P26_H05_S377_comb': 'tumor',
 'HNSCC26_P24_F12_S360_comb': 'tumor',
 'HNSCC26_P24_F03_S351_comb': 'tumor',
 'HN26_P6_D08_S44_comb': 'tumor',
 'HN25_P5_E12_S60_comb': 'Fibroblast',
 'HNSCC26_P24_C01_S313_comb': 'tumor',
 'HN25_P5_F0

In [15]:
scRNAref.columns = scRNAref.columns.map(dict_label)

In [16]:
scRNAref

,Fibroblast,Fibroblast,tumor,Fibroblast,tumor,tumor,tumor,Fibroblast,tumor,tumor,...,tumor,tumor,tumor,tumor,tumor,other,tumor,tumor,tumor,tumor
AREG,0.00000,0.00000,0.00000,0.00000,0.0000,0.44042,5.6999,0.00000,2.47270,2.78490,...,0.00000,0.30918,0.00000,0.000000,0.00000,4.96160,0.00000,3.926100,0.00000,0.00000
QPCTL,0.24853,0.72247,0.22280,0.15056,0.2302,0.22404,1.9358,0.35276,0.12168,1.27740,...,0.45838,0.75360,0.73898,0.000000,0.35840,0.86790,1.61960,0.280960,0.43082,0.66266
CNN3,7.04550,3.70750,5.28750,7.04200,4.1371,2.69890,0.0000,6.99060,3.86620,4.09200,...,4.88150,5.65310,1.87110,2.019300,0.42008,6.03100,0.00000,3.119200,0.00000,0.84157
MTRNR2L6,6.06600,6.41100,6.43460,5.23330,6.7382,4.90330,5.2151,6.41070,6.96490,6.95410,...,4.25600,3.71810,3.73100,2.707300,3.06470,5.49770,5.08650,3.745200,2.69670,2.61820
PTPN1,0.00000,0.00000,1.47510,0.00000,0.0000,2.43030,1.4163,0.29160,0.00000,0.00000,...,0.00000,1.25280,0.00000,0.000000,0.00000,4.95640,0.00000,0.875780,1.45000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOXP1,0.16993,3.49540,0.22651,4.82770,2.6567,2.64820,2.8466,2.92850,1.76980,0.00000,...,0.00000,2.49670,2.22530,0.000000,0.31034,0.00000,0.00000,0.453120,0.00000,0.00000
RABAC1,6.13160,5.22700,3.82970,0.00000,2.8266,2.66790,6.3580,5.63080,0.00000,4.64730,...,4.13770,4.50430,4.22170,0.000000,0.00000,4.62330,4.83470,3.048600,0.00000,5.19280
PTGS1,0.00000,0.00000,0.18777,0.00000,0.0000,0.02148,0.0000,0.00000,0.00000,0.00000,...,3.80180,2.66990,2.27410,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000
ZNF793,0.42438,0.99928,0.24853,0.41143,0.3115,0.64985,1.9054,0.53804,0.43936,0.57725,...,0.36065,0.31615,0.27501,0.063503,0.25822,0.59359,0.67265,0.044044,0.59264,0.33113


In [18]:
scRNAref.to_csv("/home/cke/Puram/Puram_pseudobulk_scRNAref_signature_toCIBERSORTx.tsv",sep='\t')